In [1]:
import setup

In [2]:
from model.yolo_module import YOLOModule
from data.pixta_face_datamodule import PIXTAFaceDataModule

/home/qninhdt/miniconda3/envs/cctv-od/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
import torch

pretrained = torch.load('../checkpoints/yolov7.pt', weights_only=True, map_location='cpu')

UnpicklingError: Weights only load failed. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution.Do it only if you get the file from a trusted source. WeightsUnpickler error: Unsupported class models.yolo.Model

In [3]:
model = YOLOModule.load_from_checkpoint('../checkpoints/last.ckpt')
model.to('cuda')

data = PIXTAFaceDataModule('../datasets/pixta_face/', batch_size=1, num_workers=4, pin_memory=True)
data.setup()


/home/qninhdt/miniconda3/envs/cctv-od/lib/python3.8/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [4]:
loader = iter(data.test_dataloader())

In [5]:
image, target = next(loader)

pred = model(image.to('cuda'))

In [6]:
from model.evaluators.postprocess import demo_postprocess

In [24]:
target

[{'age': tensor([3], dtype=torch.int32),
  'gender': tensor([0], dtype=torch.int32),
  'race': tensor([0], dtype=torch.int32),
  'emotion': tensor([0], dtype=torch.int32),
  'masked': tensor([0], dtype=torch.int32),
  'skintone': tensor([1], dtype=torch.int32),
  'boxes': tensor([[477.0211, 323.0415,  12.0173,  22.5171]]),
  'nboxes': tensor([[0.7547, 0.5223, 0.0188, 0.0352]])}]

In [28]:
idx = pred[:, :, 4].argmax(dim=1)
box = pred[0, idx, :4]

pred[0, idx, 5:11]

tensor([[0.0591, 0.3724, 0.3038, 0.8026, 0.8691, 0.5045]], device='cuda:0',
       grad_fn=<IndexBackward0>)